In [1]:
## This is a special client, it will try to learn from both ends

In [1]:
import random
import datetime, time
import socket, sys, cv2, numpy, time
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim as slim
import os
import csv
import datetime, time
import numpy as np
import multiprocessing
import threading
import datetime
import scipy
from scipy import signal

In [2]:
def isGreen(rgb):
    r, g, b = rgb
    if (g>r) and (g>b):
        return True
    return False

def isBlue(rgb):
    r, g, b = rgb
    if (b>r) and (b>g):
        return True
    return False

def isRed(rgb):
    r, g, b = rgb
    if (r>b) and (r>g):
        return True
    return False

In [3]:
# Copies one set of variables to another.
# Used to set worker network parameters to those of global network.
def update_target_graph(from_scope,to_scope):
    from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, from_scope)
    to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, to_scope)
    op_holder = []
    for from_var,to_var in zip(from_vars,to_vars):
        op_holder.append(to_var.assign(from_var))
    return op_holder

# Discounting function used to calculate discounted returns.
def discount(x, gamma):
    return signal.lfilter([1], [1, -gamma], x[::-1], axis=0)[::-1]

#Used to initialize weights for policy and value output layers
def normalized_columns_initializer(std=1.0):
    def _initializer(shape, dtype=None, partition_info=None):
        out = np.random.randn(*shape).astype(np.float32)
        out *= std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
        return tf.constant(out)
    return _initializer

In [4]:
def getHealths(im, coord_p1, coord_p2, right_end):
    p1 = 0
    p2 = 0
    #verde es #3CD400 == rgb(60,212,0) aproximadamente, isGreen comprueba que G sea el valor principal
    for x in range(right_end+1):
        if (isGreen(im.getpixel(coord_p1))):
            p1 = p1 + 1
        if (isGreen(im.getpixel(coord_p2))):
            p2 = p2 + 1
        coord_p1 = x, y = coord_p1[0]+1, coord_p1[1]
        coord_p2 = x, y = coord_p2[0]+1, coord_p2[1]
    return([p1,p2])

def is_game_over_neutral(HP_1, HP_2, end_time):
    if (HP_1 == 0):
        return 1 #P1 LOOSES
    elif (HP_2 == 0):
        return 2 #P2 LOOSES
    elif ((HP_1 == 0) and (HP_2 == 0)) or (datetime.datetime.now() > end_time):
        return 3 #DRAW
    return 0

def is_game_over_neutral_endless(HP_1, HP_2):
    if (HP_1 == 0):
        return 1 #P1 LOOSES
    elif (HP_2 == 0):
        return 2 #P2 LOOSES
    return 0

In [5]:
def translate_settings(buffer):
    p1_x_str = ''
    p2_x_str = ''
    p_y_str = ''
    length_str = ''
    control = 0
    for i in buffer:
        if (control == 0):
            if (i==';'):
                control +=1
            else:
                p1_x_str += i
        elif (control == 1):
            if (i==';'):
                control +=1
            else:
                p2_x_str += i
        elif (control == 2):
            if (i==';'):
                control +=1
            else:
                p_y_str += i
        elif (control == 3):
            length_str += i
    return(int(p1_x_str), int(p2_x_str), int(p_y_str), int(length_str))

In [6]:
#REGLAS: NADA DE PONERSE A ENTRENAR EN LA NOCHE ANTES DE QUE SE ACABE EL MES!!!
#SI LO HACES TE DOY UNA PALIZA
#Y SI YA TE PONES A HACERLO EN AÑO NUEVO TE MATO
def setTime():
    dt = datetime.datetime.now()
    if (dt.second % 10 > 5):
        if (dt.second>54): #Hay que pasar al siguiente minuto
            if (dt.minute==59): #Hay que pasar a la siguiente hora
                if (dt.hour == 23): #Hay que pasar al siguiente día
                    dia = dt.day+1
                    hora = 0
                    minuto = 0
                    sec = 5
                    dt.replace(day=dia, hour=hora, minute=minuto, second=sec)
                else: #hora nueva
                    hora = dt.hour+1
                    minuto = 0
                    sec = 5
                    dt.replace(hour=hora, minute=minuto, second=sec)
            else: #minuto nuevo
                minuto = dt.minute+1
                sec = 5
                dt = dt.replace(minute=minuto ,second=sec)
        else: #caso "simple"
            sec = (((dt.second // 10)+1)*10) + 5 #Z+1 5
            dt = dt.replace(second=sec)
    else:
        if (dt.second>49): #Hay que pasar al siguiente minuto
            if (dt.minute==59): #Hay que pasar a la siguiente hora
                if (dt.hour == 23): #Hay que pasar al siguiente día
                    dia = dt.day+1
                    hora = 0
                    minuto = 0
                    sec = 5
                    dt.replace(day=dia, hour=hora, minute=minuto, second=sec)
                else: #hora nueva
                    hora = dt.hour+1
                    minuto = 0
                    sec = 0
                    dt.replace(hour=hora, minute=minuto, second=sec)
            else: #minuto nuevo
                minuto = dt.minute+1
                sec = 0
                dt = dt.replace(minute=minuto ,second=sec)
        else: #caso "simple"
            sec = ((dt.second // 10)+1)*10 #Z+1 0
            dt = dt.replace(second=sec)
    return dt

In [7]:
def addMicrosecs(microsecs):
    fulldate = datetime.datetime.now()
    fulldate = fulldate + datetime.timedelta(microseconds=microsecs)
    return fulldate

def addMinute():
    fulldate = datetime.datetime.now()
    fulldate = fulldate + datetime.timedelta(minutes=1)
    return fulldate

In [8]:
def recvall(sock, count):
    buf = b''
    while count:
        newbuf = sock.recv(count)
        if not newbuf: return None
        buf += newbuf
        count -= len(newbuf)
    return buf

In [9]:
#The connection aspect of things

In [10]:
def connection_to_Monty(mensaje, player, port):
    PORT = port
    dir = "localhost"
    if (mensaje == "get"): #GET image
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((dir,PORT))
        s.sendall(mensaje.encode())
        w = int(recvall(s,8))
        h = int(recvall(s,8))
        length = recvall(s,16)
        stringData = recvall(s, int(length))
        data = numpy.fromstring(stringData, dtype='uint8').reshape(h,w,3)
        im = Image.fromarray(data)
        s.close()
        return im
    elif (mensaje == "set"): #We set the initial values of padding needed, health pos
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((dir,PORT))
        s.sendall(mensaje.encode())
        buf = s.recv(1024) #We will just need the p1_x, p2_x and y of setPadding()
        #####(x_pad, y_pad, p1_x, p2_x, y, length) = setPadding()#####
        print(buf.decode())
        s.close()
        (p1_x, p2_x, p_y, length) = translate_settings(buf.decode())
        coordinate_p1 = x, y = p1_x, p_y
        coordinate_p2 = x, y = p2_x, p_y
        return (coordinate_p1, coordinate_p2, length)
    elif (mensaje == "start1"): #START env and get wait time
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((dir,PORT))
        s.sendall(mensaje.encode())
        buf = s.recv(1024)
        s.close()
        converted = buf.decode()
        if (converted == 'Yes                 '):
            date = setTime()
            now = datetime.datetime.now()
            while(now<=date):
                time.sleep(1)
                now = datetime.datetime.now()
        return converted
    elif (mensaje == "start2"): #START env and get wait time
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((dir,PORT))
        s.sendall(mensaje.encode())
        buf = s.recv(1024)
        s.close()
        converted = buf.decode()
        if (converted == 'Yes                 '):
            date = setTime()
            now = datetime.datetime.now()
            while(now<=date):
                time.sleep(1)
                now = datetime.datetime.now()
        return converted
    else: #SEND COMMAND
        mensaje2 = player + mensaje
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((dir,PORT))
        s.sendall(mensaje2.encode())
        s.close()

In [11]:
def starter_sync(player, port):
    options = ["start1", "start2"]
    message = options[player-1]
    ip = port
    if (player == 2):
        ip -= 1
    txt = connection_to_Monty(message,str(player), ip)
    while(txt == 'No                  '):
        txt = connection_to_Monty(message,str(player), ip)
        time.sleep(1)

In [12]:
actions = ["0", "1", "2", "3", "4", "5", "6", "7", "8", 
           "01", "02", "03", "04", "05", "06", "07", "08", 
           "10", "12", "13", "14", "15", "16", "17", "18",
           "20", "21", "23", "24", "25", "26", "27", "28",
           "30", "31", "32", "34", "35", "36", "37", "38",
           "40", "41", "42", "43", "45", "46", "47", "48",
           "50", "51", "52", "53", "54", "56", "57", "58",
           "60", "61", "62", "63", "64", "65", "67", "68", 
           "70", "71", "72", "73", "74", "75", "76", "78",
           "80", "81", "82", "83", "84", "85", "86", "87"]

In [13]:
#Algunos parámetros generales
im_height = 394 #Full size
im_width = 599 #Full size
im_size_sides = im_height * im_width #Full size
im_size = im_size_sides * 3 #la anchura y altura de la imagen junto con los canales rgb #Full size

im_height_half = 197
im_width_half = 300
im_size_sides_half = im_height_half * im_width_half
im_size_half = im_size_sides_half * 3

## Creamos la red del agente (Actor-Critic Network)

In [14]:
class AC_Network():
    def __init__(self,s_size,a_size,scope,trainer):
        with tf.variable_scope(scope):
            #Input and visual encoding layers
            self.inputs = tf.placeholder(shape=[None, None,s_size],dtype=tf.float32)
            self.imageIn = tf.reshape(self.inputs,shape=[-1,im_height_half,im_width_half,3])
            self.conv1 = slim.conv2d(activation_fn=tf.nn.elu,
                inputs=self.imageIn,num_outputs=16,
                kernel_size=[8,8],stride=[4,4],padding='VALID')
            self.conv2 = slim.conv2d(activation_fn=tf.nn.elu,
                inputs=self.conv1,num_outputs=32,
                kernel_size=[4,4],stride=[2,2],padding='VALID')
            hidden = slim.fully_connected(slim.flatten(self.conv2),256,activation_fn=tf.nn.elu)
            
            #Recurrent network for temporal dependencies
            lstm_cell = tf.contrib.rnn.BasicLSTMCell(256,state_is_tuple=True)
            c_init = np.zeros((1, lstm_cell.state_size.c), np.float32)
            h_init = np.zeros((1, lstm_cell.state_size.h), np.float32)
            self.state_init = [c_init, h_init]
            c_in = tf.placeholder(tf.float32, [1, lstm_cell.state_size.c])
            h_in = tf.placeholder(tf.float32, [1, lstm_cell.state_size.h])
            self.state_in = (c_in, h_in)
            rnn_in = tf.expand_dims(hidden, [0])
            step_size = tf.shape(self.imageIn)[:1]
            state_in = tf.contrib.rnn.LSTMStateTuple(c_in, h_in)
            lstm_outputs, lstm_state = tf.nn.dynamic_rnn(
                lstm_cell, rnn_in, initial_state=state_in, sequence_length=step_size,
                time_major=False)
            lstm_c, lstm_h = lstm_state
            self.state_out = (lstm_c[:1, :], lstm_h[:1, :])
            rnn_out = tf.reshape(lstm_outputs, [-1, 256])
            
            #Output layers for policy and value estimations
            self.policy = slim.fully_connected(rnn_out,a_size,
                activation_fn=tf.nn.softmax,
                weights_initializer=normalized_columns_initializer(0.01),
                biases_initializer=None)
            self.value = slim.fully_connected(rnn_out,1,
                activation_fn=None,
                weights_initializer=normalized_columns_initializer(1.0),
                biases_initializer=None)
            
            #Only the worker network need ops for loss functions and gradient updating.
            if scope != 'global_':
                self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
                self.actions_onehot = tf.one_hot(self.actions,a_size,dtype=tf.float32)
                self.target_v = tf.placeholder(shape=[None],dtype=tf.float32)
                self.advantages = tf.placeholder(shape=[None],dtype=tf.float32)

                self.responsible_outputs = tf.reduce_sum(self.policy * self.actions_onehot, [1])

                #Loss functions
                self.value_loss = 0.5 * tf.reduce_sum(tf.square(self.target_v - tf.reshape(self.value,[-1])))
                self.entropy = - tf.reduce_sum(self.policy * tf.log(self.policy))
                self.policy_loss = -tf.reduce_sum(tf.log(self.responsible_outputs)*self.advantages)
                self.loss = 0.5 * self.value_loss + self.policy_loss - self.entropy * 0.01

                #Get gradients from local network using local losses
                local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
                self.gradients = tf.gradients(self.loss,local_vars)
                self.var_norms = tf.global_norm(local_vars)
                grads,self.grad_norms = tf.clip_by_global_norm(self.gradients,40.0)
                
                #Apply local gradients to global network
                global_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'global')
                self.apply_grads = trainer.apply_gradients(zip(grads,global_vars))

## Reward

In [15]:
def count_movements(actionBuf):
    total = 0
    divisor = 0
    for x in actionBuf:
        divisor += 1
        check = True
        for y in x:
            if (int(y)<4) and (check):
                check = False
                total +=1
    return total/divisor

def count_blocks(actionBuf):
    total = 0
    divisor = 0
    for x in actionBuf:
        divisor += 1
        for y in x:
            if y == '4':
                total += 1
    return total/divisor

In [16]:
def calculate_reward(player, status, actionBuff, current_length, hp1, hp2, flavour="vanilla"):    
    if (player == 1):
        if (flavour == "vanilla"):
            if status == 3:
                return (hp1-hp2)/200
            elif status == 0:
                return((hp1-hp2)/200)/100 #ENCOURAGEMENT
            elif status == 2:
                return -1
            else:
                return 1
        elif (flavour == "aggro"):
            if status == 3:
                return (hp1-hp2)/200
            elif status == 1:
                return (1 * (68 / current_length))
            elif status == 0:
                return((hp1-hp2)/200)/100
            else:
                return -1
        elif (flavour == "kite"):
            movements = count_movements(actionBuff)
            if status == 3:
                return ((hp1-hp2)/200)*movements
            elif status == 1:
                return 1*movements
            elif status == 0:
                return(((hp1-hp2)/200)/100)*movements
            else:
                return -1
        elif (flavour == "tank"):
            blocks = count_blocks(actionBuff)
            if status == 3:
                return ((hp1-hp2)/200)*blocks
            elif status == 1:
                return 1*blocks
            elif status == 0:
                return(((hp1-hp2)/200)/100)*blocks
            else:
                return -1
    else:
        if (flavour == "vanilla"):
            if status == 3:
                return (hp2-hp1)/200
            elif status == 0:
                return((hp2-hp1)/200)/100 #ENCOURAGEMENT
            elif status == 1:
                return -1
            else:
                return 1
        elif (flavour == "aggro"):
            if status == 3:
                return (hp2-hp1)/200
            elif status == 2:
                return (1 * (68 / current_length))
            elif status == 0:
                return((hp2-hp1)/200)/100
            else:
                return -1
        elif (flavour == "kite"):
            movements = count_movements(actionBuff)
            if status == 3:
                return ((hp2-hp1)/200)*movements
            elif status == 2:
                return 1*movements
            elif status == 0:
                return((hp2-hp1)/200)/100
            else:
                return -1
        elif (flavour == "tank"):
            blocks = count_blocks(actionBuff)
            if status == 3:
                return ((hp2-hp1)/200)*blocks
            elif status == 2:
                return 1*blocks
            elif status == 0:
                return((hp2-hp1)/200)/100
            else:
                return -1

## Worker

In [30]:
class Worker():
    def __init__(self,name,environment,player,port,s_size,a_size,trainer,model_path,global_episodes):
        self.name = "worker_" + str(name)
        self.player = player
        self.environment = environment
        self.port = port
        self.number = name        
        self.model_path = model_path
        self.trainer = trainer
        self.global_episodes = global_episodes
        self.increment = self.global_episodes.assign_add(1)
        self.episode_rewards = []
        self.episode_lengths = []
        self.episode_mean_values = []
        self.summary_writer = tf.summary.FileWriter("train_"+str(self.number))

        #Create the local copy of the network and the tensorflow op to copy global paramters to local network
        self.local_AC = AC_Network(s_size,a_size,self.name,trainer)
        self.update_local_ops = update_target_graph('global_',self.name)        
        
        self.actions = self.actions = np.identity(a_size,dtype=bool).tolist()
        
    def train(self,rollout,sess,gamma,bootstrap_value):
        rollout = np.array(rollout)
        observations = rollout[:,0]
        actions = rollout[:,1]
        rewards = rollout[:,2]
        next_observations = rollout[:,3]
        values = rollout[:,5]
        
        # Here we take the rewards and values from the rollout, and use them to 
        # generate the advantage and discounted returns. 
        # The advantage function uses "Generalized Advantage Estimation"
        self.rewards_plus = np.asarray(rewards.tolist() + [bootstrap_value])
        discounted_rewards = discount(self.rewards_plus,gamma)[:-1]
        self.value_plus = np.asarray(values.tolist() + [bootstrap_value])
        advantages = rewards + gamma * self.value_plus[1:] - self.value_plus[:-1]
        advantages = discount(advantages,gamma)

        # Update the global network using gradients from loss
        # Generate network statistics to periodically save
        feed_dict = {self.local_AC.target_v:discounted_rewards,
            self.local_AC.inputs:[np.vstack(observations)],
            self.local_AC.actions:actions,
            self.local_AC.advantages:advantages,
            self.local_AC.state_in[0]:self.batch_rnn_state[0],
            self.local_AC.state_in[1]:self.batch_rnn_state[1]}
        v_l,p_l,e_l,g_n,v_n, self.batch_rnn_state,_ = sess.run([self.local_AC.value_loss,
            self.local_AC.policy_loss,
            self.local_AC.entropy,
            self.local_AC.grad_norms,
            self.local_AC.var_norms,
            self.local_AC.state_out,
            self.local_AC.apply_grads],
            feed_dict=feed_dict)
        return v_l / len(rollout),p_l / len(rollout),e_l / len(rollout), g_n,v_n
    
    def compete(self, eps, sess, coord_p1, coord_p2, lengt):
        ep_count = 0
        total_steps = 0
        print ("Starting worker " + str(self.number) + " as player " + str(self.player) + " in env " + str(self.environment))
        with sess.as_default(), sess.graph.as_default():
             while ep_count < eps:
                sess.run(self.update_local_ops)
                episode_buffer = []
                episode_values = []
                episode_reward = 0
                episode_step_count = 0
                action_buffer = []
                d = 0
                margin = 0
                
                starter_sync(self.player, self.port)
                s = connection_to_Monty("get", str(self.player), self.port)
                wpercent = (im_width_half / float(s.size[0]))
                reduced_s = s.resize((im_width_half, im_height_half), Image.ANTIALIAS)
                s = np.reshape(np.array(reduced_s), (-1, im_size_half)) #HALF SIZE
                rnn_state = self.local_AC.state_init
                self.batch_rnn_state = rnn_state
                T1 = datetime.datetime.now()
                while (d == 0):
                    #Take an action using probabilities from policy network output.
                    a_dist,v,rnn_state = sess.run([self.local_AC.policy,self.local_AC.value,self.local_AC.state_out], 
                        feed_dict={self.local_AC.inputs:[s],
                        self.local_AC.state_in[0]:rnn_state[0],
                        self.local_AC.state_in[1]:rnn_state[1]})
                    a = np.random.choice(a_dist[0],p=a_dist[0])
                    a = np.argmax(a_dist == a)

                    connection_to_Monty(actions[a-1], str(self.player), self.port)
                    s1 = connection_to_Monty("get", str(self.player), self.port)
                    (HP_1, HP_2) = getHealths(s1, coord_p1, coord_p2, lengt)
                    margin = abs(HP_1 - HP_2)/200
                    d = is_game_over_neutral_endless(HP_1, HP_2)
                    s1_reduced = reduced_s = s1.resize((im_width_half, im_height_half), Image.ANTIALIAS)
                    s1 = np.reshape(np.array(s1_reduced), (-1, im_size_half)) #HALF SIZE

                    s = s1                    
                    total_steps += 1
                    episode_step_count += 1
                
                T2 = datetime.datetime.now()
                print("Episode: " + str(ep_count)+" ; LOOSER: " + str(d) + " ; MARGIN: " + str(margin) + " Time: " + str(T2-T1))
                if self.name == 'worker_1':
                    sess.run(self.increment)
                ep_count += 1
    
    def work(self,chosen_flavour,max_episode_length,work_eps,gamma,sess,coord,saver):
        episode_count = sess.run(self.global_episodes)
        total_steps = 0
        print ("Starting worker " + str(self.number) + " as player " + str(self.player) + " in env " + str(self.environment))
        with sess.as_default(), sess.graph.as_default():
            while episode_count <= work_eps:
                sess.run(self.update_local_ops)
                episode_buffer = []
                episode_values = []
                episode_frames = []
                episode_reward = 0
                episode_step_count = 0
                action_buffer = []
                d = 0
                
                starter_sync(self.player, self.port)
                s = connection_to_Monty("get", str(self.player), self.port)
                wpercent = (im_width_half / float(s.size[0]))
                reduced_s = s.resize((im_width_half, im_height_half), Image.ANTIALIAS)
                s = np.reshape(np.array(reduced_s), (-1, im_size_half)) #HALF SIZE
                episode_frames.append(s)
                rnn_state = self.local_AC.state_init
                self.batch_rnn_state = rnn_state
                time_limit = addMinute()
                while (datetime.datetime.now() < time_limit):
                    time_next_step = addMicrosecs(1000000/actions_per_second)
                    #Take an action using probabilities from policy network output.
                    a_dist,v,rnn_state = sess.run([self.local_AC.policy,self.local_AC.value,self.local_AC.state_out], 
                        feed_dict={self.local_AC.inputs:[s],
                        self.local_AC.state_in[0]:rnn_state[0],
                        self.local_AC.state_in[1]:rnn_state[1]})
                    a = np.random.choice(a_dist[0],p=a_dist[0])
                    a = np.argmax(a_dist == a)

                    connection_to_Monty(actions[a-1], str(self.player), self.port)
                    action_buffer.append(actions[a-1])
                    s1 = connection_to_Monty("get", str(self.player), self.port)
                    (HP_1, HP_2) = getHealths(s1, coordinate_p1[self.environment], coordinate_p2[self.environment], length[self.environment])
                    d = is_game_over_neutral(HP_1, HP_2, time_limit)  
                    r = calculate_reward(self.player, d, action_buffer, episode_step_count, HP_1, HP_2, flavour=chosen_flavour)
                    s1_reduced = reduced_s = s1.resize((im_width_half, im_height_half), Image.ANTIALIAS)
                    s1 = np.reshape(np.array(s1_reduced), (-1, im_size_half)) #HALF SIZE

                    if d == 0:
                        episode_frames.append(s1)
                    else:
                        s1 = s
                        
                    if (d == 0):
                        episode_buffer.append([s,a,r,s1,False,v[0,0]])
                    else:
                        episode_buffer.append([s,a,r,s1,True,v[0,0]])
                    episode_values.append(v[0,0])

                    episode_reward += r
                    s = s1                    
                    total_steps += 1
                    episode_step_count += 1
                    
                    # If the episode hasn't ended, but the experience buffer is full, then we
                    # make an update step using that experience rollout.
                    if len(episode_buffer) == 30 and d == 0 and episode_step_count != max_episode_length - 1:
                        # Since we don't know what the true final return is, we "bootstrap" from our current
                        # value estimation.
                        v1 = sess.run(self.local_AC.value, 
                            feed_dict={self.local_AC.inputs:[s],
                            self.local_AC.state_in[0]:rnn_state[0],
                            self.local_AC.state_in[1]:rnn_state[1]})[0,0]
                        v_l,p_l,e_l,g_n,v_n = self.train(episode_buffer,sess,gamma,v1)
                        episode_buffer = []
                        sess.run(self.update_local_ops)
                    if d != 0:
                        break
                    while(datetime.datetime.now()<time_next_step):
                        time.sleep(5.0/1000.0)
                                            
                self.episode_rewards.append(episode_reward)
                self.episode_lengths.append(episode_step_count)
                self.episode_mean_values.append(np.mean(episode_values))
                
                # Update the network using the episode buffer at the end of the episode.
                if len(episode_buffer) != 0:
                    v_l,p_l,e_l,g_n,v_n = self.train(episode_buffer,sess,gamma,0.0)
                    print("Episode: " + str(episode_count) + " ; PORT: " + str(self.port) + " ; Steps of Episode: " + str(episode_step_count) + " ; Total reward: " + str(episode_reward))   
                    
                # Periodically save gifs of episodes, model parameters, and summary statistics.
                if episode_count % 5 == 0 and episode_count != 0:
                    if episode_count % 100 == 0 and self.name == 'worker_1':
                        saver.save(sess,self.model_path+'/model-'+str(episode_count)+'.cptk')
                        print ("Saved Model")

                    mean_reward = np.mean(self.episode_rewards[-5:])
                    mean_length = np.mean(self.episode_lengths[-5:])
                    mean_value = np.mean(self.episode_mean_values[-5:])
                    summary = tf.Summary()
                    summary.value.add(tag='Perf/Reward', simple_value=float(mean_reward))
                    summary.value.add(tag='Perf/Length', simple_value=float(mean_length))
                    summary.value.add(tag='Perf/Value', simple_value=float(mean_value))
                    summary.value.add(tag='Losses/Value Loss', simple_value=float(v_l))
                    summary.value.add(tag='Losses/Policy Loss', simple_value=float(p_l))
                    summary.value.add(tag='Losses/Entropy', simple_value=float(e_l))
                    summary.value.add(tag='Losses/Grad Norm', simple_value=float(g_n))
                    summary.value.add(tag='Losses/Var Norm', simple_value=float(v_n))
                    self.summary_writer.add_summary(summary, episode_count)

                    self.summary_writer.flush()
                if self.name == 'worker_1':
                    sess.run(self.increment)
                episode_count += 1

Datos de la sesión

In [27]:
actions_per_second = 2
max_episode_length = actions_per_second * 60 #300
gamma = .99 # discount rate for advantage estimation and reward discounting
s_size = im_size_half
a_size = len(actions)
load_model = False
#model_path = './model'
#model_path = './model_AGGRO'
#model_path = './model_kite'
model_path = './model_tank'

#flavour = "vanilla"
#flavour = "aggro"
#flavour = "kite"
flavour = "tank"

PORT = 50020
work_eps = 700

In [23]:
#Solicitar los datos necesarios para el procesado de imagenes
(coordinate_p1_1, coordinate_p2_1, length_1) = connection_to_Monty("set","1", PORT)
(coordinate_p1_2, coordinate_p2_2, length_2) = connection_to_Monty("set","1", PORT+1)
coordinate_p1 = [coordinate_p1_1, coordinate_p1_2]
coordinate_p2 = [coordinate_p2_1, coordinate_p2_2]
length = [length_1, length_2]

17;376;25;200       
17;376;25;200       


## Entrenar la red

In [28]:
print(datetime.datetime.now())
time.sleep(1)
tf.reset_default_graph()

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
with tf.device("/cpu:0"): 
    global_episodes = tf.Variable(0,dtype=tf.int32,name='global_episodes',trainable=False)
    trainer = tf.train.AdamOptimizer(learning_rate=1e-4)
    master_network = AC_Network(s_size,a_size,'global_',None) # Generate global network
    num_workers = int(multiprocessing.cpu_count()/2) # Set workers to number of available CPU threads / 2, it will result in 4
    workers = []
    # Create worker classes
    for i in range(num_workers):
        workers.append(Worker(i+1,(i//2),(i%2)+1,PORT+i,s_size,a_size,trainer,model_path,global_episodes))
    saver = tf.train.Saver(max_to_keep=5)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
        
    # Start the "work" process for each worker in a separate threat.
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.work(flavour,max_episode_length,work_eps,gamma,sess,coord,saver)
        t = threading.Thread(target=(worker_work))
        t.start()
        time.sleep(0.5)
        worker_threads.append(t)
    coord.join(worker_threads)
    
print(datetime.datetime.now())

2019-05-31 09:19:36.409519
Starting worker 1 as player 1 in env 0
Starting worker 2 as player 2 in env 0
Starting worker 3 as player 1 in env 1
Starting worker 4 as player 2 in env 1


/home/tai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


Episode: 0 ; PORT: 50021 ; Steps of Episode: 139 ; Total reward: 0.0
Episode: 0 ; PORT: 50023 ; Steps of Episode: 138 ; Total reward: 0.08219130434782597
Episode: 0 ; PORT: 50020 ; Steps of Episode: 139 ; Total reward: 0.0
Episode: 0 ; PORT: 50022 ; Steps of Episode: 138 ; Total reward: -0.011504006067649153
Episode: 1 ; PORT: 50020 ; Steps of Episode: 139 ; Total reward: 0.01371630233512508
Episode: 1 ; PORT: 50022 ; Steps of Episode: 139 ; Total reward: 0.0
Episode: 1 ; PORT: 50021 ; Steps of Episode: 139 ; Total reward: -0.25121726618705037
Episode: 1 ; PORT: 50023 ; Steps of Episode: 142 ; Total reward: 0.0
Episode: 2 ; PORT: 50022 ; Steps of Episode: 140 ; Total reward: 0.0
Episode: 2 ; PORT: 50021 ; Steps of Episode: 140 ; Total reward: 0.0
Episode: 2 ; PORT: 50023 ; Steps of Episode: 141 ; Total reward: 0.0
Episode: 2 ; PORT: 50020 ; Steps of Episode: 139 ; Total reward: 0.0
Episode: 3 ; PORT: 50022 ; Steps of Episode: 138 ; Total reward: 0.0
Episode: 3 ; PORT: 50023 ; Steps of 

Episode: 26 ; PORT: 50020 ; Steps of Episode: 137 ; Total reward: 0.0
Episode: 26 ; PORT: 50021 ; Steps of Episode: 135 ; Total reward: 0.0
Episode: 27 ; PORT: 50022 ; Steps of Episode: 134 ; Total reward: 0.0
Episode: 27 ; PORT: 50023 ; Steps of Episode: 133 ; Total reward: 0.0
Episode: 27 ; PORT: 50020 ; Steps of Episode: 132 ; Total reward: 0.0
Episode: 27 ; PORT: 50021 ; Steps of Episode: 130 ; Total reward: 0.0
Episode: 28 ; PORT: 50022 ; Steps of Episode: 136 ; Total reward: 0.10543603652043107
Episode: 28 ; PORT: 50023 ; Steps of Episode: 134 ; Total reward: -0.21599402985074626
Episode: 28 ; PORT: 50021 ; Steps of Episode: 131 ; Total reward: -0.258326717557252
Episode: 28 ; PORT: 50020 ; Steps of Episode: 131 ; Total reward: 0.09704510947330562
Episode: 29 ; PORT: 50023 ; Steps of Episode: 129 ; Total reward: 0.0
Episode: 29 ; PORT: 50022 ; Steps of Episode: 130 ; Total reward: 0.0
Episode: 29 ; PORT: 50021 ; Steps of Episode: 134 ; Total reward: 0.015564179104477622
Episode: 

Episode: 54 ; PORT: 50023 ; Steps of Episode: 129 ; Total reward: -0.02482480620155039
Episode: 54 ; PORT: 50022 ; Steps of Episode: 130 ; Total reward: 0.018831088931606246
Episode: 53 ; PORT: 50021 ; Steps of Episode: 127 ; Total reward: -0.16247007874015756
Episode: 53 ; PORT: 50020 ; Steps of Episode: 128 ; Total reward: 0.10131941131725165
Episode: 55 ; PORT: 50023 ; Steps of Episode: 130 ; Total reward: 0.0
Episode: 55 ; PORT: 50022 ; Steps of Episode: 129 ; Total reward: 0.0
Episode: 54 ; PORT: 50021 ; Steps of Episode: 129 ; Total reward: -0.12945116279069765
Episode: 54 ; PORT: 50020 ; Steps of Episode: 129 ; Total reward: 0.06097325618751226
Episode: 56 ; PORT: 50023 ; Steps of Episode: 129 ; Total reward: 0.0
Episode: 56 ; PORT: 50022 ; Steps of Episode: 130 ; Total reward: 0.0
Episode: 55 ; PORT: 50021 ; Steps of Episode: 128 ; Total reward: 0.0
Episode: 55 ; PORT: 50020 ; Steps of Episode: 127 ; Total reward: 0.0
Episode: 57 ; PORT: 50023 ; Steps of Episode: 128 ; Total re

...

Episode: 577 ; PORT: 50021 ; Steps of Episode: 130 ; Total reward: 0.0
Episode: 577 ; PORT: 50020 ; Steps of Episode: 130 ; Total reward: 0.0
Episode: 582 ; PORT: 50022 ; Steps of Episode: 131 ; Total reward: 0.0
Episode: 582 ; PORT: 50023 ; Steps of Episode: 129 ; Total reward: 0.0
Episode: 578 ; PORT: 50020 ; Steps of Episode: 129 ; Total reward: -0.0220657074117963
Episode: 578 ; PORT: 50021 ; Steps of Episode: 130 ; Total reward: 0.036746153846153845
Episode: 583 ; PORT: 50023 ; Steps of Episode: 120 ; Total reward: -0.8892
Episode: 583 ; PORT: 50022 ; Steps of Episode: 119 ; Total reward: 0.2892022807500453
Episode: 579 ; PORT: 50020 ; Steps of Episode: 128 ; Total reward: 0.0
Episode: 579 ; PORT: 50021 ; Steps of Episode: 128 ; Total reward: 0.0
Episode: 584 ; PORT: 50023 ; Steps of Episode: 97 ; Total reward: -0.9155
Episode: 584 ; PORT: 50022 ; Steps of Episode: 98 ; Total reward: 0.14243827113254492
Episode: 580 ; PORT: 50020 ; Steps of Episode: 127 ; Total reward: 0.0
Episode

Episode: 603 ; PORT: 50021 ; Steps of Episode: 146 ; Total reward: 0.12240890410958903
Episode: 608 ; PORT: 50022 ; Steps of Episode: 145 ; Total reward: 0.0
Episode: 608 ; PORT: 50023 ; Steps of Episode: 146 ; Total reward: 0.0
Episode: 604 ; PORT: 50020 ; Steps of Episode: 147 ; Total reward: 0.04446344429849058
Episode: 604 ; PORT: 50021 ; Steps of Episode: 147 ; Total reward: -0.16249999999999984
Episode: 609 ; PORT: 50023 ; Steps of Episode: 99 ; Total reward: 0.24016818181818184
Episode: 609 ; PORT: 50022 ; Steps of Episode: 100 ; Total reward: -1.0100258167249978
Episode: 605 ; PORT: 50021 ; Steps of Episode: 144 ; Total reward: 0.0
Episode: 605 ; PORT: 50020 ; Steps of Episode: 142 ; Total reward: 0.0
Episode: 610 ; PORT: 50023 ; Steps of Episode: 144 ; Total reward: 0.0
Episode: 610 ; PORT: 50022 ; Steps of Episode: 144 ; Total reward: 0.0
Episode: 606 ; PORT: 50020 ; Steps of Episode: 144 ; Total reward: 0.01630718157573355
Episode: 606 ; PORT: 50021 ; Steps of Episode: 146 ;

Episode: 632 ; PORT: 50023 ; Steps of Episode: 142 ; Total reward: 0.1861028169014084
Episode: 632 ; PORT: 50022 ; Steps of Episode: 143 ; Total reward: -0.08523154423237551
Episode: 629 ; PORT: 50020 ; Steps of Episode: 139 ; Total reward: -0.9775908586184312
Episode: 629 ; PORT: 50021 ; Steps of Episode: 129 ; Total reward: 0.11926589147286826
Episode: 633 ; PORT: 50023 ; Steps of Episode: 148 ; Total reward: 0.0
Episode: 633 ; PORT: 50022 ; Steps of Episode: 146 ; Total reward: 0.0
Episode: 630 ; PORT: 50021 ; Steps of Episode: 121 ; Total reward: -0.005017355371900761
Episode: 630 ; PORT: 50020 ; Steps of Episode: 121 ; Total reward: -0.9528267854097644
Episode: 634 ; PORT: 50023 ; Steps of Episode: 139 ; Total reward: 0.4272122302158271
Episode: 634 ; PORT: 50022 ; Steps of Episode: 139 ; Total reward: -0.1822418212197907
Episode: 631 ; PORT: 50020 ; Steps of Episode: 143 ; Total reward: -0.00023686610076496134
Episode: 631 ; PORT: 50021 ; Steps of Episode: 142 ; Total reward: -0.

Episode: 654 ; PORT: 50020 ; Steps of Episode: 133 ; Total reward: 0.0
Episode: 658 ; PORT: 50023 ; Steps of Episode: 132 ; Total reward: -0.06956060606060607
Episode: 658 ; PORT: 50022 ; Steps of Episode: 130 ; Total reward: 0.05916561464401106
Episode: 659 ; PORT: 50022 ; Steps of Episode: 57 ; Total reward: -0.9638080274115134
Episode: 659 ; PORT: 50023 ; Steps of Episode: 59 ; Total reward: 0.022939830508474673
Episode: 655 ; PORT: 50021 ; Steps of Episode: 131 ; Total reward: 0.0
Episode: 655 ; PORT: 50020 ; Steps of Episode: 130 ; Total reward: 0.0
Episode: 660 ; PORT: 50023 ; Steps of Episode: 72 ; Total reward: -0.9784
Episode: 660 ; PORT: 50022 ; Steps of Episode: 61 ; Total reward: 0.21072783629427383
Episode: 656 ; PORT: 50021 ; Steps of Episode: 132 ; Total reward: 0.0
Episode: 656 ; PORT: 50020 ; Steps of Episode: 131 ; Total reward: 0.0
Episode: 661 ; PORT: 50022 ; Steps of Episode: 131 ; Total reward: 0.0
Episode: 661 ; PORT: 50023 ; Steps of Episode: 129 ; Total reward:

Episode: 684 ; PORT: 50022 ; Steps of Episode: 128 ; Total reward: 0.1656735280195531
Episode: 684 ; PORT: 50023 ; Steps of Episode: 130 ; Total reward: -0.3437538461538459
Episode: 680 ; PORT: 50021 ; Steps of Episode: 129 ; Total reward: 0.0
Episode: 680 ; PORT: 50020 ; Steps of Episode: 130 ; Total reward: 0.0
Episode: 685 ; PORT: 50023 ; Steps of Episode: 129 ; Total reward: 0.0
Episode: 685 ; PORT: 50022 ; Steps of Episode: 129 ; Total reward: 0.0
Episode: 681 ; PORT: 50021 ; Steps of Episode: 115 ; Total reward: -0.9037
Episode: 681 ; PORT: 50020 ; Steps of Episode: 115 ; Total reward: 0.20424505730814657
Episode: 686 ; PORT: 50022 ; Steps of Episode: 130 ; Total reward: -0.10448795713854195
Episode: 686 ; PORT: 50023 ; Steps of Episode: 131 ; Total reward: 0.21123206106870235
Episode: 682 ; PORT: 50021 ; Steps of Episode: 131 ; Total reward: 0.0
Episode: 682 ; PORT: 50020 ; Steps of Episode: 131 ; Total reward: 0.0
Episode: 687 ; PORT: 50022 ; Steps of Episode: 129 ; Total rewar

## Utilizar la red - P1

In [43]:
PORT = 50005
player = 1

model_path = './model'
#model_path = './model_AGGRO'
#model_path = './model_kite'
#model_path = './model_tank'

In [44]:
actions_per_second = 2.5
max_episode_length = actions_per_second * 60 #300
gamma = .99 # discount rate for advantage estimation and reward discounting
s_size = im_size_half
a_size = len(actions)
load_model = True
work_eps = 25

(coordinate_p1_1, coordinate_p2_1, length_1) = connection_to_Monty("set","1", PORT)

17;376;25;200       


In [ ]:
print(datetime.datetime.now())
time.sleep(1)
tf.reset_default_graph()

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
with tf.device("/cpu:0"): 
    global_episodes = tf.Variable(0,dtype=tf.int32,name='global_episodes',trainable=False)
    trainer = tf.train.AdamOptimizer(learning_rate=1e-4)
    master_network = AC_Network(s_size,a_size,'global_',None) # Generate global network
    num_workers = 1 # Set workers to number of available CPU threads / 2, it will result in 4
    workers = []
    # Create worker classes
    for i in range(num_workers):
        workers.append(Worker_glob(1,0,player,PORT,s_size,a_size,trainer,model_path,global_episodes))
    saver = tf.train.Saver(max_to_keep=5)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        print(ckpt.model_checkpoint_path)
        saver.restore(sess,ckpt.model_checkpoint_path)
        print ('Restored')
    else:
        sess.run(tf.global_variables_initializer())
        
    # Start the "work" process for each worker in a separate threat.
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.compete(work_eps, sess, coordinate_p1_1, coordinate_p2_1, length_1)
        t = threading.Thread(target=(worker_work))
        t.start()
        time.sleep(0.5)
        worker_threads.append(t)
    coord.join(worker_threads)

## Utilizar la red - P2

In [ ]:
PORT = 50006
player = 2

model_path = './model'
#model_path = './model_AGGRO'
#model_path = './model_kite'
#model_path = './model_tank'

In [ ]:
actions_per_second = 2.5
max_episode_length = actions_per_second * 60 #300
gamma = .99 # discount rate for advantage estimation and reward discounting
s_size = im_size_half
a_size = len(actions)
load_model = True
work_eps = 25

(coordinate_p1_1, coordinate_p2_1, length_1) = connection_to_Monty("set","1", PORT)

In [ ]:
print(datetime.datetime.now())
time.sleep(1)
tf.reset_default_graph()

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
with tf.device("/cpu:0"): 
    global_episodes = tf.Variable(0,dtype=tf.int32,name='global_episodes',trainable=False)
    trainer = tf.train.AdamOptimizer(learning_rate=1e-4)
    master_network = AC_Network(s_size,a_size,'global_',None) # Generate global network
    num_workers = 1 # Set workers to number of available CPU threads / 2, it will result in 4
    workers = []
    # Create worker classes
    for i in range(num_workers):
        workers.append(Worker(1,0,player,PORT,s_size,a_size,trainer,model_path,global_episodes))
    saver = tf.train.Saver(max_to_keep=5)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
        
    # Start the "work" process for each worker in a separate threat.
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.compete(work_eps, sess, coordinate_p1_1, coordinate_p2_1, length_1)
        t = threading.Thread(target=(worker_work))
        t.start()
        time.sleep(0.5)
        worker_threads.append(t)
    coord.join(worker_threads)